# Предсказание  следующего хода

K/k — король  

Q/q — ферзь  

R/r — ладья  

B/b — слон  

N/n — конь  

P/p — пешка  

#### Расшифровка записи

7k/5p1p/p2p1Pr1/1p4pQ/8/P1P5/2pr3P/2R2K2 w - - 0 1  

w : ход белых  

— : рокировка невозможна  

— : нет возможности взятия на проходе  

0 : 0 полуходов прошло с последнего хода пешки или взятия  

1 : первый полный ход партии (начало)  

In [2]:
path_dataset_1 = 'data/fens_training_set.csv'

path_dataset_2 = 'data/fens_processed_no_turn.csv'

## 2. Датасет

In [ ]:
from utils.dataset import ChessGNNDataset
import torch


from torch_geometric.loader import DataLoader

dataset = ChessGNNDataset(path_dataset_2)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for data in loader:
    print(data.x.shape)  # [batch_size*64, 17]
    print(data.edge_index.shape)  # [2, num_edges]
    print(data.y.shape)  # [batch_size]
    break



torch.Size([2048, 17])
torch.Size([2, 13440])
torch.Size([32])


## 3. Модель

In [ ]:
from torch_geometric.nn import GATConv, global_mean_pool
import torch.nn as nn
import torch.nn.functional as F

class GATChessGNN(nn.Module):
    def __init__(self, in_channels=17, hidden_channels=64, num_layers=2, num_classes=4096, heads=4, dropout=0.6):
        super().__init__()
        self.convs = nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels, heads=heads))
        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_channels * heads, hidden_channels, heads=heads))
        self.fc1 = nn.Linear(hidden_channels * heads, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = dropout

    def forward(self, x, edge_index, batch):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = global_mean_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        out = self.fc2(x)
        return out


In [13]:
data

DataBatch(x=[2048, 17], edge_index=[2, 13440], y=[32], batch=[2048], ptr=[33])

In [ ]:
from utils.models import ChessNetMultiInput

model = GATChessGNN(in_channels=17)

logits = model(data.x, data.edge_index, data.batch)

print(logits.shape)  # [batch_size, 4096]


torch.Size([32, 4096])


In [15]:
logits

tensor([[-0.0048, -0.0315, -0.0566,  ..., -0.0585, -0.0461, -0.0286],
        [ 0.0084, -0.0571, -0.0751,  ..., -0.0339, -0.0490, -0.0295],
        [-0.0358, -0.0776, -0.0492,  ..., -0.0233, -0.0518, -0.0013],
        ...,
        [-0.1083, -0.0679, -0.0185,  ..., -0.1084, -0.0604, -0.0379],
        [ 0.0046, -0.0442, -0.0425,  ..., -0.0439, -0.0523, -0.0012],
        [-0.0234, -0.0758, -0.0607,  ...,  0.0049, -0.0356,  0.0252]],
       grad_fn=<AddmmBackward0>)

## Тренировка

In [17]:
import torch
from torch.utils.data import Subset
from torch_geometric.loader import DataLoader

def create_gnn_data_loaders_from_csv(csv_file, batch_size=32, val_split=0.2):
    dataset = ChessGNNDataset(csv_file)

    dataset_size = len(dataset)
    indices = torch.randperm(dataset_size)
    split = int(dataset_size * (1 - val_split))

    train_indices = indices[:split]
    val_indices = indices[split:]

    train_subset = Subset(dataset, train_indices)
    val_subset = Subset(dataset, val_indices)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader


In [ ]:
train_loader, val_loader = create_gnn_data_loaders_from_csv(path_dataset_2, batch_size=2048, val_split=0.3)


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# model.load_state_dict(torch.load("save_model/graf_model.pth", map_location=torch.device(device)))  

GATChessGNN(
  (convs): ModuleList(
    (0): GATConv(17, 64, heads=4)
    (1): GATConv(256, 64, heads=4)
  )
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=4096, bias=True)
)

In [ ]:
from utils.train import train_model_gnn
train_model_gnn(model, train_loader, val_loader, device=device, epochs=50, lr=1e-1, save_path="save_model/graf_model.pth", model_name="GRAF")

Epoch 1/50


Train loss: 12.1599, acc: 0.0141
Val   loss: 6.7370, acc: 0.0188
Saved best model with val acc: 0.0188
------------------------------
Epoch 2/50


Train loss: 6.6982, acc: 0.0182
Val   loss: 6.6704, acc: 0.0188
------------------------------
Epoch 3/50


Train loss: 6.6738, acc: 0.0183
Val   loss: 6.6669, acc: 0.0188
------------------------------
Epoch 4/50


Training:  20%|██        | 26/127 [00:07<00:30,  3.31it/s, accuracy=0.0176, loss=6.66]

# Обучение медленное!!!!